In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# Load data
data_preprocessed = pd.read_csv('Absenteeism_preprocessed.csv')
data_preprocessed.head()

,reason_1,reason_2,reason_3,reason_4,Month_value,day of week,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets,Absenteeism Time in Hours
0,0,0,0,1,7,1,289,36,33,239.554,30,0,2,1,4
1,0,0,0,0,7,1,118,13,50,239.554,31,0,1,0,0
2,0,0,0,1,7,2,179,51,38,239.554,31,0,0,0,2
3,1,0,0,0,7,3,279,5,39,239.554,24,0,2,0,4
4,0,0,0,1,7,3,289,36,33,239.554,30,0,2,1,2


In [3]:
data_preprocessed['Absenteeism Time in Hours'].median()

3.0

In [4]:
targets = np.where(data_preprocessed['Absenteeism Time in Hours'] > 3, 1, 0)

In [5]:
targets.sum() / targets.shape[0]

0.45571428571428574

In [6]:
data_preprocessed['Excessive Absenteeism'] = targets
data_with_targets = data_preprocessed.drop(['Absenteeism Time in Hours', 'Daily Work Load Average', 'Distance to Work', 'day of week'], axis=1)
data_with_targets.head()

,reason_1,reason_2,reason_3,reason_4,Month_value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets,Excessive Absenteeism
0,0,0,0,1,7,289,33,30,0,2,1,1
1,0,0,0,0,7,118,50,31,0,1,0,0
2,0,0,0,1,7,179,38,31,0,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0,1
4,0,0,0,1,7,289,33,30,0,2,1,0


In [7]:
unscaled_inputs = data_with_targets.iloc[:, :-1]
unscaled_inputs.head()

,reason_1,reason_2,reason_3,reason_4,Month_value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,7,289,33,30,0,2,1
1,0,0,0,0,7,118,50,31,0,1,0
2,0,0,0,1,7,179,38,31,0,0,0
3,1,0,0,0,7,279,39,24,0,2,0
4,0,0,0,1,7,289,33,30,0,2,1


In [8]:


class CustomScaler(BaseEstimator,TransformerMixin): 
    def __init__(self,columns):
        self.scaler = StandardScaler()
        self.columns = columns
        self.mean_ = None
        self.var_ = None
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self

    def transform(self, X, y=None, copy=None):
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [9]:
unscaled_inputs.columns.values

array(['reason_1', 'reason_2', 'reason_3', 'reason_4', 'Month_value',
       'Transportation Expense', 'Age', 'Body Mass Index', 'Education',
       'Children', 'Pets'], dtype=object)

In [10]:
# columns_to_scale = ['Month_value', 'day of week', 'Transportation Expense', 'Distance to Work', 'Age',
#        'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pets']

columns_to_omit = ['reason_1', 'reason_2', 'reason_3', 'reason_4', 'Education']

In [11]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [12]:
absenteeism_scaler = CustomScaler(columns_to_scale)
absenteeism_scaler.fit(unscaled_inputs)

scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)
scaled_inputs

,reason_1,reason_2,reason_3,reason_4,Month_value,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
1,0,0,0,0,0.182726,-1.574681,2.130803,1.002633,0,-0.019280,-0.589690
2,0,0,0,1,0.182726,-0.654143,0.248310,1.002633,0,-0.919030,-0.589690
3,1,0,0,0,0.182726,0.854936,0.405184,-0.643782,0,0.880469,-0.589690
4,0,0,0,1,0.182726,1.005844,-0.536062,0.767431,0,0.880469,0.268487
...,...,...,...,...,...,...,...,...,...,...,...
695,1,0,0,0,-0.388293,-0.654143,0.562059,-1.114186,1,0.880469,-0.589690
696,1,0,0,0,-0.388293,0.040034,-1.320435,-0.643782,0,-0.019280,1.126663
697,1,0,0,0,-0.388293,1.624567,-1.320435,-0.408580,1,-0.919030,-0.589690
698,0,0,0,1,-0.388293,0.190942,-0.692937,-0.408580,1,-0.919030,-0.589690


In [13]:
# splitting into train and test set


x_train, x_test,  y_train, y_test = train_test_split(scaled_inputs, targets, train_size=0.8, random_state=20)

In [14]:
print(x_train.shape, y_train.shape)

(560, 11) (560,)


In [15]:
print(x_test.shape, y_test.shape)

(140, 11) (140,)


In [16]:
# Model: Logistic Regression


reg = LogisticRegression()
reg.fit(x_train, y_train)

reg.score(x_train, y_train)

0.7732142857142857

In [17]:
reg.intercept_

array([-1.6474549])

In [18]:
reg.coef_

array([[ 2.80019733,  0.95188356,  3.11555338,  0.83900082,  0.1589299 ,
         0.60528415, -0.16989096,  0.27981088, -0.21053312,  0.34826214,
        -0.27739602]])

In [19]:
feature_name = unscaled_inputs.columns.values
summary_table = pd.DataFrame(columns = ['Feature_Name'], data = feature_name)
summary_table['Coefficient'] = np.transpose(reg.coef_)
summary_table

,Feature_Name,Coefficient
0,reason_1,2.800197
1,reason_2,0.951884
2,reason_3,3.115553
3,reason_4,0.839001
4,Month_value,0.158930
5,Transportation Expense,0.605284
6,Age,-0.169891
7,Body Mass Index,0.279811
8,Education,-0.210533
9,Children,0.348262


In [20]:
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()
summary_table

,Feature_Name,Coefficient
0,Intercept,-1.647455
1,reason_1,2.800197
2,reason_2,0.951884
3,reason_3,3.115553
4,reason_4,0.839001
5,Month_value,0.158930
6,Transportation Expense,0.605284
7,Age,-0.169891
8,Body Mass Index,0.279811
9,Education,-0.210533


In [21]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)
summary_table

,Feature_Name,Coefficient,Odds_ratio
0,Intercept,-1.647455,0.192539
1,reason_1,2.800197,16.447892
2,reason_2,0.951884,2.590585
3,reason_3,3.115553,22.545903
4,reason_4,0.839001,2.314054
5,Month_value,0.158930,1.172256
6,Transportation Expense,0.605284,1.831773
7,Age,-0.169891,0.843757
8,Body Mass Index,0.279811,1.322880
9,Education,-0.210533,0.810152


In [22]:
summary_table.sort_values('Odds_ratio', ascending=False)

,Feature_Name,Coefficient,Odds_ratio
3,reason_3,3.115553,22.545903
1,reason_1,2.800197,16.447892
2,reason_2,0.951884,2.590585
4,reason_4,0.839001,2.314054
6,Transportation Expense,0.605284,1.831773
10,Children,0.348262,1.416604
8,Body Mass Index,0.279811,1.322880
5,Month_value,0.158930,1.172256
7,Age,-0.169891,0.843757
9,Education,-0.210533,0.810152


In [23]:
reg.score(x_test, y_test)

0.75

In [24]:
predict_proba = reg.predict_proba(x_test)

In [25]:
predict_proba[:, 1]

array([0.28659587, 0.41275772, 0.55979179, 0.21840536, 0.91589146,
       0.66512397, 0.70015424, 0.86896029, 0.21374596, 0.25096368,
       0.50602402, 0.77515087, 0.92870849, 0.26821867, 0.69065865,
       0.4528329 , 0.44947725, 0.4607293 , 0.59798883, 0.94638425,
       0.2996991 , 0.21840536, 0.57962872, 0.57962872, 0.75216435,
       0.25433741, 0.48982726, 0.14309805, 0.79650267, 0.21840536,
       0.36956558, 0.67906035, 0.68502567, 0.52868083, 0.21840536,
       0.53506551, 0.22147081, 0.73692105, 0.40498044, 0.60505988,
       0.21075848, 0.45224466, 0.23751292, 0.39833498, 0.82755447,
       0.56797575, 0.69113325, 0.28659587, 0.21935267, 0.2033097 ,
       0.57628256, 0.3294664 , 0.66512397, 0.26949499, 0.83321968,
       0.43491525, 0.88374612, 0.23127072, 0.33415858, 0.34432939,
       0.69909345, 0.65494263, 0.29244941, 0.79200758, 0.20750276,
       0.26840558, 0.08708566, 0.22147081, 0.73245417, 0.30530219,
       0.22147081, 0.29014408, 0.90438191, 0.46061297, 0.60174

In [26]:
# saving model


with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [27]:
with open('scaler', 'wb') as file:
    pickle.dump(absenteeism_scaler, file)

In [28]:
scaled_inputs['reason_2'].sum()

6